<img src="https://imagedelivery.net/Dr98IMl5gQ9tPkFM5JRcng/3e5f6fbd-9bc6-4aa1-368e-e8bb1d6ca100/Ultra" alt="Image description" width="160" />

Introduction to Contextual AI Platform

The Contextual APIs provide a simple interface to our state-of-the-art Contextual Language Models (CLMs). Use this guide to learn the basics of how to create your first application programmatically. In this demo, we will be tuning and deploying a CLM.

To run this notebook interactively, you can open it in Google Colab:

<a target="_blank" href="https://colab.research.google.com/github/ContextualAI/ContextualAI-Examples/blob/main/python/tune-api-example.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

### API Key
To begin, you will need an API key to securely access the API. Please contact Contextual's sales team to get your API key.

In [11]:
CONTEXTUAL_API_KEY="key-..."

In [12]:
import os
from contextual import ContextualAI

# create a client
client = ContextualAI(
    api_key=CONTEXTUAL_API_KEY,
)

# test the API Key
try:
    response = create_agent_output = client.agents.list()
    print("Valid API Key.")
except Exception as e:
    print(f"Invalid API Key: {e}")

Valid API Key.


### Create an Agent

You will need to first create an agent. An agent allows to group together a dataset and queries, as well as add in prompt guidelines. Everything you'll do following this will reference this agent. 

In [13]:
# Create an agent with name 'My First Agent'
try:
    create_agent_output = client.agents.create(
        name="My First Agent"
    )
    print(create_agent_output.model_dump_json())
    agent_id = create_agent_output.id
except Exception as e:
    print(f"Encountered error: {e}")

{"id":"c530ddd6-e9bc-45a3-8786-47ff703c7307","datastore_ids":["5aca8364-7064-452e-a572-e74cd416de8e"]}


### Create a Datastore

A datastore securely stores the files you would like to search and reference in your agents. Each agent must be associated with at least one datastore. You can create a datastore using the /datastores endpoint with the following command:



In [14]:
import requests

def download_file_from_gdrive(file_id, output_file):
   # Direct download URL format for Google Drive
   url = f"https://drive.google.com/uc?export=download&id={file_id}"

   response = requests.get(url)
   with open(output_file, 'wb') as f:
       f.write(response.content)

# Extract file ID from the URL
file_id = "1MlsscmU9gtbCrZk3C_CNbCDvEkQ7AhUM"

# Download the file
download_file_from_gdrive(file_id, "dataset.jsonl")

### Tune a model

Create a tuning job for the specified Agent. Tuning jobs are asynchronous tasks to specialize your Agent to your specific domain or use case.

This API initiates a tuning specialization task using the provided `training_file` and an optional `test_file`. If no test_file is provided, the tuning job will hold out a portion of the training_file as the test set.

Returns a tune job id which can be used to check on the status of your tuning task through the `client.agents.tune.jobs.metadata` command.

After the tune job completes, the metadata associated with the tune job will include evaluation results and a model ID.

In [15]:
# now call the update dataset API
try:
    with open('dataset.jsonl', 'rb') as f:
        response = dataset_metadata = client.agents.tune.create(
            agent_id=agent_id,
            training_file=f,
        )
        job_id = response.id
        print(response.to_dict())
except Exception as e:
    print(f"Encountered error: {e}")

Encountered error: Error code: 422 - {'detail': "Validation error: Error in item 0, field 'reference': Field required"}


## Check the status


You can check the status of the job using the API. For more information, please see the API docs. When the job is complete, the status will turn to completed. The response payload will also contain evaluation_results .

In [16]:
# now call the update dataset API
try:
    response = client.agents.tune.jobs.metadata(
        job_id=job_id,
    )

    print(response.to_dict())
except Exception as e:
    print(f"Encountered error: {e}")

Encountered error: name 'job_id' is not defined


When the job is complete, the metadata would look like the following:
```
{'job_status': 'completed',
 'evaluation_results': {'grounded_generation_train_test.json_equivalence': 1.0,
  'grounded_generation_train_test.json_helpfulness': 0.814156498263641,
  'grounded_generation_train_test.json_groundedness': 0.7781168677598632},
 'model_id': 'registry/model-ada3c484-3ce0f31f-llm-fd6c2'}
 ```

In [7]:
model_id = response.model_id
model_id

'registry/model-ada3c484-3ce0f31f-llm-fd6c2'

### Update the Application

Once the tuned job is complete, you can deploy the tuned model via editing the application through API. Note that currently a single fine-tuned model deployment is allowed per tenant. Please see the API doc for more information.

In [ ]:
def update_application(application_id: str, llm_model_id: str):
   """Update application's LLM model"""
   url = f"{BASE_URL}/applications/{application_id}"
   headers = {
       "accept": "application/json",
       "content-type": "application/json",
       "Authorization": f"Bearer {API_TOKEN}"
   }
   data = {"llm_model_id": llm_model_id}
   response = requests.put(url, headers=headers, json=data)
   return response.json()

In [ ]:
#update_application(APPLICATION_ID, model_id)
update_application('3ce0f31f-b262-4de6-aad8-411c5016b4ac', model_id)